In [1]:
import pandas as pd

import re
import preprocessor as p
from html import unescape
from urlextract import URLExtract
from bs4 import BeautifulSoup

In [2]:
df = pd.read_csv('twcs.csv')

In [3]:
df.head()

,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id
0,1,sprintcare,False,Tue Oct 31 22:10:47 +0000 2017,@115712 I understand. I would like to assist y...,2,3.0
1,2,115712,True,Tue Oct 31 22:11:45 +0000 2017,@sprintcare and how do you propose we do that,NaN,1.0
2,3,115712,True,Tue Oct 31 22:08:27 +0000 2017,@sprintcare I have sent several private messag...,1,4.0
3,4,sprintcare,False,Tue Oct 31 21:54:49 +0000 2017,@115712 Please send us a Private Message so th...,3,5.0
4,5,115712,True,Tue Oct 31 21:49:35 +0000 2017,@sprintcare I did.,4,6.0


In [4]:
print(f"df_shape: {df.shape}",
      f"unique_text: {df.text.unique().size}")

df_shape: (2811774, 7) unique_text: 2782618


In [5]:
df_clients = df[df['inbound'] == True]

In [6]:
df_clients.head()

,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id
1,2,115712,True,Tue Oct 31 22:11:45 +0000 2017,@sprintcare and how do you propose we do that,NaN,1.0
2,3,115712,True,Tue Oct 31 22:08:27 +0000 2017,@sprintcare I have sent several private messag...,1,4.0
4,5,115712,True,Tue Oct 31 21:49:35 +0000 2017,@sprintcare I did.,4,6.0
6,8,115712,True,Tue Oct 31 21:45:10 +0000 2017,@sprintcare is the worst customer service,"9,6,10",NaN
8,12,115713,True,Tue Oct 31 22:04:47 +0000 2017,@sprintcare You gonna magically change your co...,"11,13,14",15.0


In [7]:
print(f"df_clients_shape: {df_clients.shape}")

df_clients_shape: (1537843, 7)


In [8]:
df_clients_content = df_clients[['tweet_id', 'text']]

In [9]:
df_clients_content.head()

,tweet_id,text
1,2,@sprintcare and how do you propose we do that
2,3,@sprintcare I have sent several private messag...
4,5,@sprintcare I did.
6,8,@sprintcare is the worst customer service
8,12,@sprintcare You gonna magically change your co...


In [10]:
# clean the dataset
def clean_tweets(column):
    # punctuations to be replaced
    REPLACE_NO_SPACE = re.compile("(\;)|(\:)|(\!)|(\')|(\,)|(\")|(\|)|(\()|(\))|(\[)|(\])|(\%)|(\$)|(\>)|(\<)|(\{)|(\})")
    REPLACE_WITH_SPACE = re.compile("(<br\s/><br\s/?)|(-)|(/)|(:).")
    tempArr = []
    for line in column:
        tmpL = p.clean(line)
        tmpL = REPLACE_NO_SPACE.sub("", tmpL.lower())
        tmpL = REPLACE_WITH_SPACE.sub(" ", tmpL)
        tempArr.append(tmpL)
    return tempArr

In [11]:
df_clients_content['clean_text'] = clean_tweets(df_clients_content['text'])

C:\Users\skryp\AppData\Local\Temp\ipykernel_8980\407807772.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clients_content['clean_text'] = clean_tweets(df_clients_content['text'])


In [12]:
df_clients_content.loc[5:100]

,tweet_id,text,clean_text
6,8,@sprintcare is the worst customer service,is the worst customer service
8,12,@sprintcare You gonna magically change your co...,you gonna magically change your connectivity f...
10,16,@sprintcare Since I signed up with you....Sinc...,since i signed up with you....since day
12,18,@115714 y’all lie about your “great” connectio...,yall lie about your great connection. bars lte...
14,20,"@115714 whenever I contact customer support, t...",whenever i contact customer support they tell ...
16,22,@Ask_Spectrum Would you like me to email you a...,would you like me to email you a copy of one s...
18,26,@Ask_Spectrum I received this from your corpor...,i received this from your corporate office wou...
20,23,@Ask_Spectrum The correct way to do it is via ...,the correct way to do it is via an ocs account...
21,24,@Ask_Spectrum That is INCORRECT information I ...,that is incorrect information i have the form ...
23,29,actually that's a broken link you sent me and ...,actually thats a broken link you sent me and i...


In [13]:
prepared_dataset = df_clients_content[['tweet_id', 'clean_text']]

In [14]:
prepared_dataset.head()

,tweet_id,clean_text
1,2,and how do you propose we do that
2,3,i have sent several private messages and no on...
4,5,i did.
6,8,is the worst customer service
8,12,you gonna magically change your connectivity f...


In [16]:
prepared_dataset.to_csv('twcs_prepared.csv', index=False)